In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wfdb
import stumpy

import os

In [2]:

annotations = []
ecgs = []

for file_index in range(1,10):
    #file_name = files.split(".")[0]
    ecg = wfdb.rdsamp('apnea/apnea/a0'+str(file_index))
    ecgs.append(ecg[0].reshape(len(ecg[0])))
    annotation = wfdb.rdann('apnea/apnea/a0'+str(file_index), extension="apn")
    annotations.append(annotation.symbol)
for file_index in range(10,21):
    #file_name = files.split(".")[0]
    ecg = wfdb.rdsamp('apnea/apnea/a'+str(file_index))
    ecgs.append(ecg[0].reshape(len(ecg[0])))
    annotation = wfdb.rdann('apnea/apnea/a'+str(file_index), extension="apn")
    annotations.append(annotation.symbol)

for file_index in range(1,6):
    #file_name = files.split(".")[0]
    ecg = wfdb.rdsamp('apnea/apnea/b0'+str(file_index))
    ecgs.append(ecg[0].reshape(len(ecg[0])))
    annotation = wfdb.rdann('apnea/apnea/b0'+str(file_index), extension="apn")
    annotations.append(annotation.symbol)

for file_index in range(1,10):
    #file_name = files.split(".")[0]
    ecg = wfdb.rdsamp('apnea/apnea/c0'+str(file_index))
    ecgs.append(ecg[0].reshape(len(ecg[0])))
    annotation = wfdb.rdann('apnea/apnea/c0'+str(file_index), extension="apn")
    annotations.append(annotation.symbol)
for file_index in range(10,11):
    #file_name = files.split(".")[0]
    ecg = wfdb.rdsamp('apnea/apnea/c'+str(file_index))
    ecgs.append(ecg[0])
    annotation = wfdb.rdann('apnea/apnea/c'+str(file_index), extension="apn")
    annotations.append(annotation.symbol)

In [3]:
## simplify ecg signal by getting one sample per second (reduced from 100 samples/second)

In [4]:
simple_ecgs = []
for ecg in ecgs:
    simple_ecgs.append(ecg.reshape(-1, 100).mean(axis=1))

In [5]:
## Proposed Algorithm 
def extract_stationary_indexes(t, window_size, sliding_window, break_points, round_index):
    matrix_profile = stumpy.stump(np.asarray(t,dtype=float), m=window_size)
    df_matrix_profile = pd.DataFrame(matrix_profile)

    
    df_matrix_profile['left_index']= matrix_profile[:,1]
    proposed_change_points = []

    for x in range(0,len(df_matrix_profile.index)-1):
            if df_matrix_profile['left_index'].iloc[x+1] != df_matrix_profile['left_index'].iloc[x] + 1:
                proposed_change_points.append(x)

    
    variance_changes = []
    final_indexes = []
    for x in proposed_change_points:
        if x < sliding_window:
            continue
        elif x > len(t) - sliding_window:
            continue
        else:
            final_indexes.append(x)
            window_mean = np.mean([t[y] for y in range(x-sliding_window, x)])/np.std([t[y] for y in range(x-sliding_window, x)])
            next_window_mean = np.mean([t[y] for y in range(x,x+sliding_window)])/np.std([t[y] for y in range(x,x+sliding_window)])
        
        if x < sliding_window:
            continue
        elif x > len(t) - sliding_window:
            continue
        else:
            window_std = np.std([t[y] for y in range(x-sliding_window, x)])
            next_window_std = np.std([t[y] for y in range(x,x+sliding_window)])
        
            mean_change = np.abs(window_mean - next_window_mean)
            std_change = np.abs(window_std - next_window_std)
            variance_changes.append(mean_change * std_change)
            #variance_changes.append(mean_change)

    df_entropy_changes = pd.DataFrame()
    #df_entropy_changes['indexes'] = proposed_change_points
    df_entropy_changes['indexes'] = final_indexes
    df_entropy_changes['relative_variance'] = variance_changes

    df_entropy_changes = df_entropy_changes[df_entropy_changes['relative_variance'] != 0]
    
    df_entropy_changes['rounded_indexes'] = [round(x,round_index) for x in df_entropy_changes['indexes']]
    df_sorted = df_entropy_changes.loc[df_entropy_changes.groupby('rounded_indexes').relative_variance.idxmax()]
    df_entropy_sorted = df_sorted.sort_values(by='relative_variance')

    selected_indexes = []
    for x in df_entropy_sorted['indexes'].iloc[len(df_entropy_sorted['rounded_indexes'])-break_points: len(df_entropy_sorted['rounded_indexes'])]:
        selected_indexes.append(x)
    return selected_indexes

In [ ]:
ground_truth_indexes = []
my_indexes = []
flussy_indexes = []
import ruptures as rpt
ruptures_indexes = []

for i in range(0,len(simple_ecgs)):
    print(i)
    labels = annotations[i]
    signal = simple_ecgs[i]
    
    
    number_regimes = 1
    N_indexes = []

    for label_index in range(0,len(labels)-1):
        if labels[label_index] != labels[label_index + 1]:
            number_regimes += 1
            N_indexes.append((label_index+1)*60)
    
    ground_truth_indexes.append(N_indexes)
    my_indexes.append(extract_stationary_indexes(signal,60,300,number_regimes-1,-3))
    m = 60
    mp = stumpy.stump(signal, m=m)
    L = 60
    cac, regime_locations = stumpy.fluss(mp[:, 1], L=L, n_regimes=number_regimes, excl_factor=5)
    flussy_indexes.append(regime_locations)
    print("here")
    algo = rpt.Dynp(model="l2").fit(signal)
    result = algo.predict(n_bkps=number_regimes)
    ruptures_indexes.append(result)
    
    """model = "rbf"  # "l1", "rbf", "linear", "normal", "ar"
    algo = rpt.Window(width=60, model=model).fit(signal)
    my_bkps = algo.predict(epsilon=1)
    ruptures2_indexes.append(my_bkps)"""


0
here


In [ ]:
ruptures_indexes

In [ ]:
ground_truth_indexes

In [ ]:
my_indexes

In [ ]:
flussy_indexes

In [ ]:
plt.plot(simple_ecgs[0])
for index in [4513, 25252, 3467, 4317, 13695]:
    plt.axvline(index,color='magenta')
for index in [12058, 11748, 11411, 12358, 10958]:
    plt.axvline(index,color='yellow')
for index in [780, 13620, 13860, 21120, 21240]:
    plt.axvline(index,color='green')

In [ ]:
plt.plot(simple_ecgs[1])
for index in [27560,
  22214,
  21048,
  7493,
  486,
  19514,
  4476,
  7811,
  554,
  31416,
  16705,
  26133,
  10296,
  13038,
  15303,
  19300,
  15604,
  12116,
  30080,
  10905,
  1989]:
    plt.axvline(index,color='magenta')
for index in [1749,  8549, 10766,  8178,  8941,  6618, 10465,  3415, 11066,
         9720,  1446,  7453,  7767,  6147,  6993, 10134,  9241,  3788,
         5796, 11447, 11775]:
    plt.axvline(index,color='yellow')
for index in [1260,
  7800,
  10560,
  12060,
  12420,
  15960,
  16200,
  16920,
  17280,
  17700,
  17820,
  18480,
  18720,
  19560,
  19740,
  23340,
  23520,
  25920,
  26160,
  30000,
  30540]:
    plt.axvline(index,color='green')

In [ ]:
plt.plot(simple_ecgs[2][10:])
for index in [16690,
  28215,
  20828,
  25576,
  4147,
  18125,
  29740,
  6329,
  9004,
  26734,
  14045,
  14679,
  15786,
  30944,
  13436,
  29302,
  24247,
  11255,
  24956,
  11593,
  300]:
    plt.axvline(index,color='magenta')
for index in [12791, 12215, 11289, 11674, 13091, 13494, 15981, 13794, 14193,
        15283, 14932, 15628, 16416, 14493, 10965, 17658, 17266, 18008,
        20906, 18376, 16716]:
    plt.axvline(index,color='yellow')
for index in [780,
  3600,
  6300,
  8820,
  11160,
  13740,
  14400,
  15480,
  17400,
  18180,
  20820,
  21180,
  23100,
  24360,
  24660,
  25020,
  25620,
  26580,
  28200,
  29520,
  30420]:
    plt.axvline(index,color='green')
    
plt.savefig('apneasegmentation.eps', format='eps')

In [ ]:
for i in range(0,len(my_indexes)):
    my_index = my_indexes[i].sort()
    flussy_index = flussy_indexes[i].sort()


In [ ]:
for i in range(0,len(ground_truth_indexes)):
    ground_truth_indexes[i].sort()
    

In [ ]:
print(ground_truth_indexes[0])
print(my_indexes[0])

In [ ]:
my_samples_hits = []
my_total_true_cp = 0
my_total_predicted_cp = 0
wrong_score = 0
for sample in range(0,len(my_indexes)):
    sample_hits = 0
    tolerance = len(simple_ecgs[sample])/10
    ground_truth = ground_truth_indexes[sample]
    if len(ground_truth) == len(my_indexes[sample]):
        my_total_true_cp += len(ground_truth)
        my_total_predicted_cp += len(my_indexes[sample])
        for i in range(0,len(ground_truth)):
            if my_indexes[sample][i] in np.arange(ground_truth[i]-tolerance,ground_truth[i]+tolerance):
                sample_hits = sample_hits + 1
        my_samples_hits.append(sample_hits)
    else:
        wrong_score += 1
print(wrong_score)        
my_samples_hits

In [ ]:
flussy_true_cp = 0
flussy_predicted_cp = 0
flussy_samples_hits = []
wrong_score = 0
for sample in range(0,len(flussy_indexes)):
    sample_hits = 0
    tolerance = len(simple_ecgs[sample])/10
    ground_truth = ground_truth_indexes[sample]
    if len(ground_truth) == len(flussy_indexes[sample]):
        flussy_true_cp += len(ground_truth)
        flussy_predicted_cp += len(flussy_indexes[sample])
        for i in range(0,len(ground_truth)):
            if flussy_indexes[sample][i] in np.arange(ground_truth[i]-tolerance,ground_truth[i]+tolerance):
                sample_hits = sample_hits + 1
        flussy_samples_hits.append(sample_hits)
    else:
        wrong_score += 1
print(wrong_score)        
flussy_samples_hits


In [ ]:
my_samples_precision = sum(my_samples_hits)/my_total_predicted_cp

my_samples_recall = sum(my_samples_hits)/my_total_true_cp


my_samples_fscore = 2 * my_samples_precision * my_samples_recall/(my_samples_precision + my_samples_recall)

my_samples_fscore

In [ ]:
flussy_samples_precision = sum(flussy_samples_hits)/flussy_predicted_cp

flussy_samples_recall = sum(flussy_samples_hits)/flussy_true_cp


flussy_samples_fscore = 2 * flussy_samples_precision * flussy_samples_recall/(flussy_samples_precision + flussy_samples_recall)

flussy_samples_fscore

In [ ]:
my_samples_hausdorff = []
wrong_score = 0
for sample in range(0,len(my_indexes)):
    max_distance = 0
    ground_truth = ground_truth_indexes[sample]
    if len(ground_truth) == len(my_indexes[sample]):
        for i in range(0,len(ground_truth)):
            if np.abs(my_indexes[sample][i]-ground_truth[i]) > max_distance:
                max_distance = np.abs(my_indexes[sample][i]-ground_truth[i])
        my_samples_hausdorff.append(max_distance/len(simple_ecgs[sample]))
    else:
        wrong_score += 1
print(wrong_score)        
np.mean(my_samples_hausdorff)

In [ ]:
flussy_samples_hausdorff = []
wrong_score = 0
for sample in range(0,len(flussy_indexes)):
    max_distance = 0
    ground_truth = ground_truth_indexes[sample]
    if len(ground_truth) == len(flussy_indexes[sample]):
        for i in range(0,len(ground_truth)):
            if np.abs(flussy_indexes[sample][i]-ground_truth[i]) > max_distance:
                max_distance = np.abs(flussy_indexes[sample][i]-ground_truth[i])
        flussy_samples_hausdorff.append(max_distance/len(simple_ecgs[sample]))
    else:
        wrong_score += 1
print(wrong_score)        
np.mean(flussy_samples_hausdorff)